# MLP model

## Cài đặt và nâng cấp các thư viện cần thiết

In [ ]:
# Cài đặt TensorFlow Federated
!pip install --upgrade tensorflow_federated

# Cài đặt TensorFlow và các thư viện cần thiết
!pip install tensorflow tensorflow_datasets pandas scikit-learn

## MLP - FedAvg

In [ ]:
# Mô hình MLP sử dụng thuật toán Federated Learning - FedAvg
# Cài đặt thư viện cần thiết
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Đọc dữ liệu
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv', sep=';')

# Tách dữ liệu
X = df.drop('output', axis=1).values.astype(np.float32)
y = df['output'].values.astype(np.float32)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Tạo tf.data.Dataset cho client
def create_tf_dataset(features, labels):
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(16)

def create_keras_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

def create_federated_data(X, y, num_clients=5):
    client_data = []
    data_splits = np.array_split(np.column_stack((X, y)), num_clients)
    for split in data_splits:
        feats = split[:, :-1]
        labels = split[:, -1]
        client_data.append(create_tf_dataset(feats.astype(np.float32), labels.astype(np.float32)))
    return client_data

federated_train_data = create_federated_data(X_train, y_train, num_clients=5)
input_spec = federated_train_data[0].element_spec

# Khởi tạo mô hình
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )


# Tạo trình tối ưu theo chuẩn mới của TFF
client_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=0.01)
server_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=1.0)

# Khởi tạo quá trình Federated Averaging (3.1. Lựa chọn thuật toán phù hợp FedAvg)
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer,
    server_optimizer_fn=server_optimizer
)

state = iterative_process.initialize()

# Huấn luyện mô hình Federated Learning (3.3. Huấn luyện mô hình theo Federated learning)
for round_num in range(1, 51):
    result = iterative_process.next(state, federated_train_data)
    state = result.state
    print(f"Federated Round {round_num}: {result.metrics}")

# Lấy trọng số từ mô hình liên kết
final_model_weights = iterative_process.get_model_weights(state)

# Tạo lại mô hình keras và gán trọng số
fed_model = create_keras_model()
final_model_weights.assign_weights_to(fed_model)

# Dự đoán
y_pred_fed = (fed_model.predict(X_test) > 0.5).astype("int32").flatten()

# Đánh giá
print("\nĐánh giá mô hình học liên kết (FedAvg):")
print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_fed))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred_fed))
print("Độ đặc hiệu (Specificity - Negative):", recall_score(y_test, y_pred_fed, pos_label=0))
print("Độ chính xác theo lớp (Precision):", precision_score(y_test, y_pred_fed))
print("Ma trận nhầm lẫn:\n", confusion_matrix(y_test, y_pred_fed))

# So sánh với mô hình học tập trung (Centralized Learning)
central_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
central_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
central_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

# Dự đoán và đánh giá mô hình (3.4. Đánh giá mô hình)
y_pred = (central_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học tập trung:")
print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred))
print("Độ đặc hiệu (Specificity - Negative):", recall_score(y_test, y_pred, pos_label=0))
print("Độ chính xác theo lớp (Precision):", precision_score(y_test, y_pred))
print("Ma trận nhầm lẫn:\n", confusion_matrix(y_test, y_pred))

ERROR:jax._src.xla_bridge:Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 428, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_r

Mounted at /content/drive
Federated Round 1: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.57024795), ('loss', 0.718468), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 2: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5661157), ('loss', 0.709878), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 3: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.5785124), ('loss', 0.7017189), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', Ordered

In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 200 -> 0
sample = np.array([[62,	1,	0,	120,	267,	0,	1,	99,	1,	1.8,	1,	2,	3]], dtype=np.float32)
sample_scaled = scaler.transform(sample)
fed_prediction = fed_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học liên kết - FedAvg")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 76ms/step

Dự đoán bằng mô hình học liên kết - FedAvg
Xác suất bị đau tim: 0.2168
Kết luận: Nguy cơ thấp
1/1 [==============================] - 0s 52ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.0656
Kết luận: Nguy cơ thấp


In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 2 -> 1
sample = np.array([[63,	1,	3,	145,	233,	1,	0,	150,	0,	2.3,	0,	0, 1]], dtype=np.float32)
sample_scaled = scaler.transform(sample)
fed_prediction = fed_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học liên kết - FedAvg")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 59ms/step

Dự đoán bằng mô hình học liên kết - FedAvg
Xác suất bị đau tim: 0.8122
Kết luận: Nguy cơ cao
1/1 [==============================] - 0s 154ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.6065
Kết luận: Nguy cơ cao


## MLP - FedProx

In [ ]:
# Cài đặt thư viện
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from google.colab import drive

# Kết nối Google Drive
drive.mount('/content/drive')

# Đọc dữ liệu
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv', sep=';')

# Tiền xử lý dữ liệu
X = df.drop('output', axis=1).values.astype(np.float32)
y = df['output'].values.astype(np.float32)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Chia dữ liệu cho client
def create_tf_dataset(features, labels):
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(16)

def create_keras_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

def create_federated_data(X, y, num_clients=5):
    client_data = []
    splits = np.array_split(np.column_stack((X, y)), num_clients)
    for split in splits:
        feats = split[:, :-1]
        labels = split[:, -1]
        client_data.append(create_tf_dataset(feats, labels))
    return client_data

federated_train_data = create_federated_data(X_train, y_train, num_clients=5)
input_spec = federated_train_data[0].element_spec

# Mô hình cho TFF
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Sử dụng FedProx (cần TFF >= 0.53+ để hỗ trợ build_weighted_fed_prox)
client_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=0.01)
server_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=1.0)

iterative_process = tff.learning.algorithms.build_weighted_fed_prox(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer,
    server_optimizer_fn=server_optimizer,
    proximal_strength=0.1  # Hệ số proximal lambda (tùy chỉnh theo thử nghiệm)
)

state = iterative_process.initialize()

# Huấn luyện mô hình Federated Learning với FedProx
for round_num in range(1, 51):
    result = iterative_process.next(state, federated_train_data)
    state = result.state
    print(f"Federated Round {round_num}: {result.metrics}")

# Trích xuất trọng số mô hình cuối cùng
final_model_weights = iterative_process.get_model_weights(state)

# Tạo lại mô hình keras để gán trọng số
fed_model = create_keras_model()
final_model_weights.assign_weights_to(fed_model)

# Đánh giá mô hình Federated
y_pred_fed = (fed_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học liên kết (FedProx):")
print("Độ chính xác:", accuracy_score(y_test, y_pred_fed))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred_fed))
print("Độ đặc hiệu (Recall - Negative):", recall_score(y_test, y_pred_fed, pos_label=0))
print("Precision:", precision_score(y_test, y_pred_fed))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_fed))

# So sánh với mô hình tập trung
central_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
central_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
central_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

y_pred = (central_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học tập trung:")
print("Độ chính xác:", accuracy_score(y_test, y_pred))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred))
print("Độ đặc hiệu (Recall - Negative):", recall_score(y_test, y_pred, pos_label=0))
print("Precision:", precision_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Federated Round 1: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.32231405), ('loss', 0.80855256), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 2: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.338843), ('loss', 0.7934198), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 3: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.36363637), ('loss', 0.77942246), ('num_examples', 242), ('num_batches

In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 200 -> 0
sample = np.array([[62,	1,	0,	120,	267,	0,	1,	99,	1,	1.8,	1,	2,	3]], dtype=np.float32)
sample_scaled = scaler.transform(sample)
fed_prediction = fed_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học liên kết - FedProx")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 21ms/step

Dự đoán bằng mô hình học liên kết - FedProx
Xác suất bị đau tim: 0.2771
Kết luận: Nguy cơ thấp
1/1 [==============================] - 0s 20ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.0507
Kết luận: Nguy cơ thấp


In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 2 -> 1
sample = np.array([[63,	1,	3,	145,	233,	1,	0,	150,	0,	2.3,	0,	0, 1]], dtype=np.float32)
sample_scaled = scaler.transform(sample)
fed_prediction = fed_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học liên kết - FedProx")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 22ms/step

Dự đoán bằng mô hình học liên kết - FedProx
Xác suất bị đau tim: 0.6070
Kết luận: Nguy cơ cao
1/1 [==============================] - 0s 20ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.7372
Kết luận: Nguy cơ cao


# CNN Model

## Cài đặt thư viện cần thiết

In [ ]:
!pip install --quiet tensorflow_federated nest-asyncio

## CNN - FedAvg

In [ ]:
# Mô hình CNN sử dụng thuật toán Federated Learning - FedAvg
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Đọc dữ liệu
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv', sep=';')

# Tiền xử lý dữ liệu
X = df.drop('output', axis=1).values.astype(np.float32)
y = df['output'].values.astype(np.float32)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_reshaped = X_scaled.reshape(-1, X.shape[1], 1, 1)  # CNN reshape
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# Tạo tf.data.Dataset cho client
def create_tf_dataset(features, labels):
    return tf.data.Dataset.from_tensor_slices((features, labels)).shuffle(100).batch(16)

def create_keras_model():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 1), activation='relu', input_shape=(X.shape[1], 1, 1)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

def create_federated_data(X, y, num_clients=5):
    client_data = []
    data_splits = np.array_split(np.column_stack((X.reshape(len(X), -1), y)), num_clients)
    for split in data_splits:
        feats = split[:, :-1].reshape(-1, X.shape[1], 1, 1)
        labels = split[:, -1]
        client_data.append(create_tf_dataset(feats.astype(np.float32), labels.astype(np.float32)))
    return client_data

federated_train_data = create_federated_data(X_train, y_train, num_clients=5)
input_spec = federated_train_data[0].element_spec

# Khởi tạo mô hình
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Tạo trình tối ưu
client_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=0.01)
server_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=1.0)

# Khởi tạo FedAvg
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer,
    server_optimizer_fn=server_optimizer
)

state = iterative_process.initialize()

# Huấn luyện mô hình
for round_num in range(1, 51):
    result = iterative_process.next(state, federated_train_data)
    state = result.state
    print(f"Federated Round {round_num}: {result.metrics}")

# Lấy trọng số mô hình
final_model_weights = iterative_process.get_model_weights(state)
fed_model = create_keras_model()
final_model_weights.assign_weights_to(fed_model)

# Dự đoán và đánh giá mô hình học liên kết
y_pred_fed = (fed_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học liên kết (FedAvg - CNN):")
print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred_fed))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred_fed))
print("Độ đặc hiệu (Specificity - Negative):", recall_score(y_test, y_pred_fed, pos_label=0))
print("Độ chính xác theo lớp (Precision):", precision_score(y_test, y_pred_fed))
print("Ma trận nhầm lẫn:\n", confusion_matrix(y_test, y_pred_fed))

# Học tập trung để so sánh
central_model = create_keras_model()
central_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
central_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

y_pred = (central_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học tập trung (CNN):")
print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred))
print("Độ đặc hiệu (Specificity - Negative):", recall_score(y_test, y_pred, pos_label=0))
print("Độ chính xác theo lớp (Precision):", precision_score(y_test, y_pred))
print("Ma trận nhầm lẫn:\n", confusion_matrix(y_test, y_pred))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Federated Round 1: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.661157), ('loss', 0.66797733), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 2: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.677686), ('loss', 0.6642922), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 3: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.6983471), ('loss', 0.6611493), ('num_examples', 242), ('num_batches', 1

In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 200 -> 0
sample = np.array([[62,	1,	0,	120,	267,	0,	1,	99,	1,	1.8,	1,	2,	3]], dtype=np.float32)
sample_scaled_cnn = np.expand_dims(sample_scaled, axis=-1)      # (1, 13, 1)
sample_scaled_cnn = np.expand_dims(sample_scaled_cnn, axis=-1)  # (1, 13, 1, 1)

fed_prediction = fed_model.predict(sample_scaled_cnn)
print(f"\nDự đoán bằng mô hình học liên kết - FedAvg")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled_cnn)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 20ms/step

Dự đoán bằng mô hình học liên kết - FedAvg
Xác suất bị đau tim: 0.4646
Kết luận: Nguy cơ thấp
1/1 [==============================] - 0s 46ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.6535
Kết luận: Nguy cơ cao


## CNN - FedProx

In [ ]:
# Cài đặt thư viện
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix
from google.colab import drive

# Kết nối Google Drive
drive.mount('/content/drive')

# Đọc dữ liệu
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/heart.csv', sep=';')

# Tiền xử lý dữ liệu
X = df.drop('output', axis=1).values.astype(np.float32)
y = df['output'].values.astype(np.float32)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Chia dữ liệu cho client
def create_tf_dataset(features, labels):
    return tf.data.Dataset.from_tensor_slices((features, labels)).batch(16)

# Cập nhật mô hình CNN
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((X.shape[1], 1), input_shape=(X.shape[1],)),  # Reshape dữ liệu thành dạng (samples, features, 1)
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),  # Lớp convolutional 1D
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Lớp max pooling
        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),  # Lớp convolutional thứ hai
        tf.keras.layers.MaxPooling1D(pool_size=2),  # Lớp max pooling thứ hai
        tf.keras.layers.Flatten(),  # Chuyển đổi đầu ra của các lớp convolutional thành dạng phẳng
        tf.keras.layers.Dense(64, activation='relu'),  # Lớp fully connected
        tf.keras.layers.Dense(32, activation='relu'),  # Lớp fully connected
        tf.keras.layers.Dense(1, activation='sigmoid')  # Lớp đầu ra cho bài toán phân loại nhị phân
    ])
    return model


def create_federated_data(X, y, num_clients=5):
    client_data = []
    splits = np.array_split(np.column_stack((X, y)), num_clients)
    for split in splits:
        feats = split[:, :-1]
        labels = split[:, -1]
        client_data.append(create_tf_dataset(feats, labels))
    return client_data

federated_train_data = create_federated_data(X_train, y_train, num_clients=5)
input_spec = federated_train_data[0].element_spec

# Mô hình cho TFF
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Sử dụng FedProx (cần TFF >= 0.53+ để hỗ trợ build_weighted_fed_prox)
client_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=0.01)
server_optimizer = tff.learning.optimizers.build_sgdm(learning_rate=1.0)

iterative_process = tff.learning.algorithms.build_weighted_fed_prox(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer,
    server_optimizer_fn=server_optimizer,
    proximal_strength=0.1  # Hệ số proximal lambda (tùy chỉnh theo thử nghiệm)
)

state = iterative_process.initialize()

# Huấn luyện mô hình Federated Learning với FedProx
for round_num in range(1, 51):
    result = iterative_process.next(state, federated_train_data)
    state = result.state
    print(f"Federated Round {round_num}: {result.metrics}")

# Trích xuất trọng số mô hình cuối cùng
final_model_weights = iterative_process.get_model_weights(state)

# Tạo lại mô hình keras để gán trọng số
fed_model = create_keras_model()
final_model_weights.assign_weights_to(fed_model)

# Đánh giá mô hình Federated
y_pred_fed = (fed_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học liên kết (FedProx):")
print("Độ chính xác:", accuracy_score(y_test, y_pred_fed))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred_fed))
print("Độ đặc hiệu (Recall - Negative):", recall_score(y_test, y_pred_fed, pos_label=0))
print("Precision:", precision_score(y_test, y_pred_fed))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_fed))

# So sánh với mô hình tập trung
central_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
central_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
central_model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

y_pred = (central_model.predict(X_test) > 0.5).astype("int32").flatten()

print("\nĐánh giá mô hình học tập trung:")
print("Độ chính xác:", accuracy_score(y_test, y_pred))
print("Độ nhạy (Recall - Positive):", recall_score(y_test, y_pred))
print("Độ đặc hiệu (Recall - Negative):", recall_score(y_test, y_pred, pos_label=0))
print("Precision:", precision_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Federated Round 1: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.446281), ('loss', 0.7018749), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 2: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.44214877), ('loss', 0.70098835), ('num_examples', 242), ('num_batches', 17)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Federated Round 3: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('binary_accuracy', 0.42561984), ('loss', 0.7001334), ('num_examples', 242), ('num_batches'

2/2 [==============================] - 0s 6ms/step

Đánh giá mô hình học tập trung:
Độ chính xác: 0.8688524590163934
Độ nhạy (Recall - Positive): 0.84375
Độ đặc hiệu (Recall - Negative): 0.896551724137931
Precision: 0.9
Confusion Matrix:
 [[26  3]
 [ 5 27]]


In [ ]:
# age	sex	cp	trtbps	chol	fbs	restecg	thalachh	exng	oldpeak	slp	caa	thall	output, dự đoán với dòng 2 -> 1
sample = np.array([[63,	1,	3,	145,	233,	1,	0,	150,	0,	2.3,	0,	0, 1]], dtype=np.float32)
sample_scaled = scaler.transform(sample)
fed_prediction = fed_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học liên kết - FedProx")
print(f"Xác suất bị đau tim: {fed_prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if fed_prediction[0][0] > 0.5 else "Nguy cơ thấp")

prediction = central_model.predict(sample_scaled)
print(f"\nDự đoán bằng mô hình học tập trung")
print(f"Xác suất bị đau tim: {prediction[0][0]:.4f}")
print("Kết luận:", "Nguy cơ cao" if prediction[0][0] > 0.5 else "Nguy cơ thấp")

1/1 [==============================] - 0s 31ms/step

Dự đoán bằng mô hình học liên kết - FedProx
Xác suất bị đau tim: 0.5539
Kết luận: Nguy cơ cao
1/1 [==============================] - 0s 57ms/step

Dự đoán bằng mô hình học tập trung
Xác suất bị đau tim: 0.7004
Kết luận: Nguy cơ cao
